In [1]:
import glob
import cv2
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
from random import shuffle

# Parameters

In [2]:
input_data_path = "Dataset/**/*.jpg" 
trash_type = ['compost', 'landfill', 'recyclable'] # Keep 3 types only
shuffle_data = True
tvt = {'train': 0.6, 'val': 0.8, 'test': 1} # Train: 60%, Val: 20%, Test: 20%

train_filename = 'train.tfrecords'
val_filename = 'val.tfrecords'
test_filename = 'test.tfrecords'
resize_image = (224, 224)
resize_image_set = [224, 224, 3]

IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CHANNELS = 3 # RGB

# Split dataset into training, validation & test set.

In [3]:
def split_data(input_data_path, trash_type, shuffle_data, tvt, return_type):

    labels = []
    
    # Labelling data
    files = glob.glob(input_data_path)
    for file in files:
        if trash_type[0] in file:
            labels.append(0)
        elif trash_type[1] in file:
            labels.append(1)
        elif trash_type[2] in file:
            labels.append(2)
        else:
            labels.append(-1)

    # Shuffling data
    if shuffle_data:
        c = list(zip(files, labels))
        shuffle(c)
        files, labels = zip(*c)

    # Dividing data into 60% train, 20% validation, and 20% test
    train_files = files[0:int(tvt['train']*len(files))]
    train_labels = labels[0:int(tvt['train']*len(files))]
    val_files = files[int(tvt['train']*len(files)):int(tvt['val']*len(files))]
    val_labels = labels[int(tvt['train']*len(files)):int(tvt['val']*len(files))]
    test_files = files[int(tvt['val']*len(files)):]
    test_labels = labels[int(tvt['val']*len(files)):]
    
    if return_type == 'train':
        return train_files, train_labels
    elif return_type == 'val':
        return val_files, val_labels
    elif return_type == 'test':
        return test_files, test_labels

# Write to tfrecords file

In [4]:
def load_image(addr):
    # Read an image and resizing it.
    # cv2 loads images as BGR; converting it to RGB
    img = Image.open(addr)
    img = cv2.imread(addr)
    img = cv2.resize(img, resize_image, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def write_to_tfrecords(filename, files, label, tvt):
    
    # Open .tfrecords file
    writer = tf.python_io.TFRecordWriter(filename)
    
    for i in range(len(files)):
    
        # Load image and its label
        img = load_image(train_files[i])
        label = train_labels[i]

        # Create a feature
        feature = { tvt+'/label': _int64_feature(label),
                    tvt+'/image': _bytes_feature(tf.compat.as_bytes(img.tobytes()))}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))

        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

# Read from tfrecords file

In [5]:
# Read from .tfrecords file
def read_from_tfrecords(tvt):
    
    data_path = glob.glob(tvt+'.tfrecords')

    feature = { tvt+'/image': tf.FixedLenFeature([], tf.string),
                tvt+'/label': tf.FixedLenFeature([], tf.int64)}

    # Create list of all .tfrecords files and pass it to queue
    filename_queue = tf.train.string_input_producer(data_path, num_epochs=1)

    # Define reader and read file from queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)

    # Convert serialized data back to arrays and numbers
    image = tf.decode_raw(features[tvt+'/image'], tf.float32)
    label = tf.cast(features[tvt+'/label'], tf.int32)

    # Reshape image data into the original shape
    image = tf.reshape(image, resize_image_set)

    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=5, capacity=10, 
                                        num_threads=1, min_after_dequeue=5)

    return images, labels

# CNN

In [6]:
def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    
    # Setup filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                      num_filters]

    # Initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # Setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')

    # Add the bias
    out_layer += bias

    # Apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)

    # Perform max pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, 
                               padding='SAME')

    return out_layer

# Main

In [7]:
train_files, train_labels = split_data(input_data_path, trash_type, shuffle_data, tvt, 'train')
val_files, val_labels = split_data(input_data_path, trash_type, shuffle_data, tvt, 'val')
test_files, test_labels = split_data(input_data_path, trash_type, shuffle_data, tvt, 'test')

write_to_tfrecords(train_filename, train_files, train_labels, 'train')
write_to_tfrecords(val_filename, val_files, val_labels, 'val')
write_to_tfrecords(test_filename, test_files, test_labels, 'test')

image_batch, label_batch = read_from_tfrecords('train')

In [10]:
# # Create CNN
# # Python optimisation variables
# learning_rate = 0.0001
# epochs = 10
# batch_size = 5

# # Declare the training data placeholders
# # Input x - for 224 x 224 x 3 = 150528 - this is the flattened image data that is drawn from shuffle_batch()
# x = tf.placeholder(tf.float32, [None, 150528])

# # Dynamically reshape the input
# x_shaped = tf.reshape(x, [-1, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])

# # Declare the output data placeholder - 3 labels (compost, landfill & recyclable)
# y = tf.placeholder(tf.float32, [None, 3])